In [0]:
!nvcc --version
!cat /etc/issue
import tensorflow as tf
from platform import python_version

print(python_version())
print(tf.__version__)
import torch
print(torch.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130
Ubuntu 18.04.3 LTS \n \l



3.6.9
1.15.0
1.3.1


In [0]:
%cd /content
import os
from google.colab import drive
drive.mount('/content/gdrive')

/content
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
project_path = '/content/gdrive/My Drive' #change dir to your project folder
os.chdir(project_path)  #change dir

In [0]:
!git clone https://github.com/mingyuliutw/UNIT.git

fatal: destination path 'UNIT' already exists and is not an empty directory.


In [0]:
# %cd /content
!ls

cuda-repo-ubuntu1804-10-2-local-10.2.89-440.33.01_1.0-1_amd64.deb
model
nv-tensorrt-repo-ubuntu1804-cuda10.0-trt5.1.2.2-rc-20190227_1-1_amd64.deb
nv-tensorrt-repo-ubuntu1804-cuda10.2-trt6.0.1.8-ga-20191108_1-1_amd64.deb
PytorchToTensorRT.ipynb
TensorRT-5.1.2.2
TensorRT-5.1.2.2.Ubuntu-18.04.1.x86_64-gnu.cuda-10.0.cudnn7.5.tar.gz
TensorRT-6.0.1.5
torch2trt
UNIT
UNIT_Working


In [0]:
# !tar xzvf TensorRT-5.1.2.2.Ubuntu-18.04.1.x86_64-gnu.cuda-10.0.cudnn7.5.tar.gz

In [0]:
!ls TensorRT-5.1.2.2/

data  graphsurgeon  python   TensorRT-Release-Notes.pdf
doc   include	    targets  uff


In [0]:
os.environ['LD_LIBRARY_PATH'] += os.pathsep + '/content/gdrive/My Drive/PytorchToTensorRT/TensorRT-5.1.2.2/lib'

In [0]:
%cd /content/gdrive/My Drive/PytorchToTensorRT/TensorRT-5.1.2.2/python
!ls

/content/gdrive/My Drive/PytorchToTensorRT/TensorRT-5.1.2.2/python
tensorrt-5.1.2.2-cp27-none-linux_x86_64.whl
tensorrt-5.1.2.2-cp34-none-linux_x86_64.whl
tensorrt-5.1.2.2-cp35-none-linux_x86_64.whl
tensorrt-5.1.2.2-cp36-none-linux_x86_64.whl
tensorrt-5.1.2.2-cp37-none-linux_x86_64.whl


In [0]:
!pip3 install tensorrt-5.1.2.2-cp36-none-linux_x86_64.whl

Processing ./tensorrt-5.1.2.2-cp36-none-linux_x86_64.whl


In [0]:
%cd /content/gdrive/My Drive/PytorchToTensorRT/TensorRT-5.1.2.2/uff
!ls

/content/gdrive/My Drive/PytorchToTensorRT/TensorRT-5.1.2.2/uff
uff-0.6.3-py2.py3-none-any.whl


In [0]:
!pip3 install uff-0.6.3-py2.py3-none-any.whl

Processing ./uff-0.6.3-py2.py3-none-any.whl


In [0]:
%cd /content/gdrive/My Drive/PytorchToTensorRT/TensorRT-5.1.2.2/graphsurgeon
!ls

/content/gdrive/My Drive/PytorchToTensorRT/TensorRT-5.1.2.2/graphsurgeon
graphsurgeon-0.4.0-py2.py3-none-any.whl


In [0]:
!pip3 install graphsurgeon-0.4.0-py2.py3-none-any.whl

Processing ./graphsurgeon-0.4.0-py2.py3-none-any.whl


In [0]:
!apt-get update
!apt-get install -y --no-install-recommends libnvinfer5=5.1.2-1+cuda10.0
!apt-get install -y --no-install-recommends libnvinfer-dev=5.1.2-1+cuda10.0
!apt-get install python3-libnvinfer-dev

In [0]:
# %cd /content
# !git clone https://github.com/NVIDIA-AI-IOT/torch2trt
# %cd torch2trt
# !sudo python setup.py install

In [0]:
os.environ['PYTHONPATH'] += os.pathsep + '/content/gdrive/My Drive/UNIT/'#PytorchToTensorRT/UNIT_Working'

In [0]:
!pip install torchfile

  Created wheel for torchfile: filename=torchfile-0.1.0-cp36-none-any.whl size=5711 sha256=81d42c94650bc968b25af875b0ab660a38d9963005f26bb8e85847033b97f281
  Stored in directory: /root/.cache/pip/wheels/b1/c3/d6/9a1cc8f3a99a0fc1124cae20153f36af59a6e683daca0a0814
Successfully built torchfile


In [0]:
project_path = '/content/gdrive/My Drive/PytorchToTensorRT/UNIT_Working' #change dir to your project folder
# project_path = '/content/gdrive/My Drive/UNIT' #change dir to your project folder
os.chdir(project_path)  #change dir

In [0]:
from __future__ import print_function
from utils import get_config, get_data_loader_folder
from trainer import MUNIT_Trainer, UNIT_Trainer
import argparse
from torch.autograd import Variable
import torchvision.utils as vutils
import sys
import torch
import os
from torchvision import transforms
from PIL import Image

In [0]:
torch.manual_seed(0)
torch.cuda.manual_seed(0)

# Load experiment setting
config = get_config('configs/unit_n2d_folder.yaml')
# opts.num_style = 1 if opts.style != '' else opts.num_style

# Setup model and data loader
config['vgg_model_path'] = './output'
trainer = UNIT_Trainer(config)

state_dict = torch.load('checkpoints/gen_01017000.pt')
trainer.gen_a.load_state_dict(state_dict['a'])
trainer.gen_b.load_state_dict(state_dict['b'])
trainer.cuda()
trainer.eval()
trainer.train(False)
encode = trainer.gen_a # encode function
style_encode = trainer.gen_b.encode # encode function
decode = trainer.gen_b # decode function

In [0]:
trainer.state_dict

In [0]:
# print(encode)
onnx_path = "/content/gdrive/My Drive/result.onnx"
# model = trainer
# model = models.densenet121(pretrained=True)
# model = model.to('cuda')
# export_onnx_model(decode, input_shape, onnx_path)
input1 = torch.ones((1, 3, 256, 256)).cuda()
input2 = torch.ones((1, 3, 256, 256)).cuda()
# inputs = Variable(torch.randn(1, 3, 256, 256)).cuda()
print("==============ooooooo==============================")
# model(inputs)
print("==============kkkkkkkk==============================")
# {'a': self.gen_a.state_dict(), 'b': self.gen_b.state_dict()}
torch.onnx.export(trainer, args = (input1, input2) , f = onnx_path)

==============ooooooo==============================
==============kkkkkkkk==============================


/content/gdrive/My Drive/PytorchToTensorRT/UNIT_Working/trainer.py:245: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  x_a.volatile = True
/content/gdrive/My Drive/PytorchToTensorRT/UNIT_Working/trainer.py:246: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  x_b.volatile = True
/content/gdrive/My Drive/PytorchToTensorRT/UNIT_Working/networks.py:285: TracerWarning: There are 2 live references to the data region being modified when tracing in-place operator add_. This might cause the trace to be incorrect, because all other views that also reference this data will not reflect this change in the trace! On the other hand, if all other views use the same memory chunk, but are disjoint (e.g. are outputs of torch.split), this might still be safe.
  out += residual
/usr/local/lib/python3.6/dist-packages/torch/onnx/symbolic_helper.py:198: UserWarning: You are trying to export the model with onnx:Ups

In [0]:
cd UNIT_Working//

/content/gdrive/My Drive/PytorchToTensorRT/UNIT_Working


In [0]:
import torch
torch.__version__
state_dict = torch.load('checkpoints/gen_01017000.pt')

In [0]:
!pip install torchfile

  Created wheel for torchfile: filename=torchfile-0.1.0-cp36-none-any.whl size=5711 sha256=2def4923ba2681414088e5aa5103bcc489b128eb0798fc1ec28612df30953a1c
  Stored in directory: /root/.cache/pip/wheels/b1/c3/d6/9a1cc8f3a99a0fc1124cae20153f36af59a6e683daca0a0814
Successfully built torchfile


In [0]:
!python test_batch.py --trainer UNIT --config configs/unit_n2d_folder.yaml  --input input/ --output_folder output/ --checkpoint checkpoints/gen_01017000.pt --a2b 1

input/b1c81faa-3df17267.jpg
test_batch.py:103: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  images = Variable(images.cuda(), volatile=True)
index:  0 time:  0.15184473991394043
size:  torch.Size([1, 3, 500, 888])
average FPS:  6.585674291824401
input/b1c81faa-c80764c5.jpg
index:  1 time:  0.11034107208251953
size:  torch.Size([1, 3, 500, 888])
average FPS:  9.062808445907987
input/b1ca2e5d-84cf9134.jpg
index:  2 time:  0.08021831512451172
size:  torch.Size([1, 3, 500, 888])
average FPS:  12.465981097307258
input/b1ca8418-84a133a0.jpg
index:  3 time:  0.07901525497436523
size:  torch.Size([1, 3, 500, 888])
average FPS:  12.655784004296741
input/b1cd1e94-26dd524f.jpg
index:  4 time:  0.07523417472839355
size:  torch.Size([1, 3, 500, 888])
average FPS:  13.29183185181664
input/b1ceb32e-3f481b43.jpg
index:  5 time:  0.07967782020568848
size:  torch.Size([1, 3, 500, 888])
average FPS:  12.5505441466458
input/b1ceb32e-813f84b2.jpg
index:  6 